<a href="https://colab.research.google.com/github/Sim98B/MushroomVision/blob/main/MushD4/MushDetector4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mushroom Detector

## Setup

In [1]:
# Importing main libraries
try:
  import torch
  import torchvision
  import torchinfo
except:
  print("[INFO] Torch and Torchvision not installed, downloading")
  !pip install -q torch torchvision torchinfo
import torch
print(f"torch: {torch.__version__}")
import torchvision
print(f"torchvision: {torchvision.__version__}")
import torchinfo
print(f"torchinfo: {torchinfo.__version__}")

# Utilities
import os
from pathlib import Path
import requests
import shutil
from timeit import default_timer as timer
import warnings
warnings.filterwarnings("ignore")

# Manipulation
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt

# Neural networks
from torch import nn
from torchvision import transforms
import torchvision.models
from torchinfo import summary

# Metrics
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Cloning GitHub repository
!git clone https://github.com/Sim98B/MushroomVision
!mv MushroomVision/Modules/Libraries .

# Custom functions
from Libraries import data, extractors, model_ops, utils

shutil.rmtree("MushroomVision")

[INFO] Torch and Torchvision not installed, downloading
torch: 2.1.0+cu118
torchvision: 0.16.0+cu118
torchinfo: 1.8.0
Cloning into 'MushroomVision'...
remote: Enumerating objects: 3841, done.
remote: Counting objects: 100% (586/586), done.
remote: Compressing objects: 100% (341/341), done.
remote: Total 3841 (delta 279), reused 470 (delta 214), pack-reused 3255
Receiving objects: 100% (3841/3841), 580.58 MiB | 36.79 MiB/s, done.
Resolving deltas: 100% (968/968), done.
Updating files: 100% (1739/1739), done.


In [2]:
# Device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Data: CIFAR100

In [3]:
tensor_transform = transforms.Compose([
    transforms.ToTensor(),
])

train_data = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform = tensor_transform)
test_data = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform = tensor_transform)

print(f"{len(train_data)} images for train divided into {len(train_data.classes)} classes")
print(f"{len(test_data)} images for train divided into {len(test_data.classes)} classes")

100%|██████████| 169001437/169001437 [00:04<00:00, 34223799.48it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
50000 images for train divided into 100 classes
10000 images for train divided into 100 classes


In [4]:
for i in range(len(train_data.classes)):
  if train_data.classes[i] == "mushroom":
    print(f"{i+1}. {train_data.classes[i].upper()}")

class_names = train_data.classes

52. MUSHROOM


## Setting up experiments

In [5]:
ALEXNET, alexnet_transformer = extractors.create_alexnet(output_shape = len(class_names))
VGG, vgg_transformer = extractors.create_vgg16(output_shape = len(class_names))
DENSENET, densenet_transformer = extractors.create_densenet121(output_shape = len(class_names))
RESNET, resnet_transformer = extractors.create_resnet50(output_shape = len(class_names))

for model in [ALEXNET, VGG, DENSENET, RESNET]:
  if model.__class__.__name__ != "ResNet":
    print(f"{model.__class__.__name__} params:\nTOTAL: {summary(model, input_size = (32, 3, 256, 256)).total_params:,}\nTRAINABLE: {summary(model, input_size = (32, 3, 256, 256)).trainable_params:,}")
  else:
    print(f"{model.__class__.__name__} params:\nTOTAL: {summary(model, input_size = (32, 3, 232, 232)).total_params:,}\nTRAINABLE: {summary(model, input_size = (32, 3, 232, 232)).trainable_params:,}")

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 84.3MB/s]
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 79.5MB/s]
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 91.1MB/s]
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 148MB/s]


AlexNet params:
TOTAL: 57,413,540
TRAINABLE: 409,700
VGG params:
TOTAL: 134,670,244
TRAINABLE: 409,700
DenseNet params:
TOTAL: 7,056,356
TRAINABLE: 102,500
ResNet params:
TOTAL: 23,712,932
TRAINABLE: 204,900


In [ ]:
utils.set_seed()

experiment_number = 0

for epoch in [5, 10, 15, 20]:

  for extractor in [ALEXNET, VGG, DENSENET, RESNET]:
    experiment_number += 1
    print(f"Experiment n°{experiment_number}, model: {extractor.__class__.__name__}, {epoch} epochs")

    if extractor == ALEXNET:

      model, transformer = extractors.create_alexnet(output_shape = len(class_names))
      train_data.transform = transformer
      test_data.transform = transformer
      experiment_train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                                            batch_size = 32,
                                                            shuffle = True,
                                                            num_workers = os.cpu_count(),
                                                            pin_memory = True)
      experiment_val_loader = torch.utils.data.DataLoader(dataset = test_data,
                                                          batch_size = 32,
                                                          shuffle = False,
                                                          num_workers = os.cpu_count(),
                                                          pin_memory = True)

      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001)

      model_ops.train(model = model,
                      train_dataloader = experiment_train_loader,
                      test_dataloader = experiment_val_loader,
                      loss_function = criterion,
                      optimizer = optimizer,
                      metric = "accuracy",
                      epochs = epoch,
                      device = device,
                      writer = utils.create_writer(experiment_name = f"{epoch}_epochs",
                                                   model_name = extractor.__class__.__name__),
                      verbose = 2)

    elif extractor == VGG:

      model, transformer = extractors.create_vgg16(output_shape = len(class_names))
      train_data.transform = transformer
      test_data.transform = transformer
      experiment_train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                                            batch_size = 32,
                                                            shuffle = True,
                                                            num_workers = os.cpu_count(),
                                                            pin_memory = True)
      experiment_val_loader = torch.utils.data.DataLoader(dataset = test_data,
                                                          batch_size = 32,
                                                          shuffle = False,
                                                          num_workers = os.cpu_count(),
                                                          pin_memory = True)

      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001)

      model_ops.train(model = model,
                      train_dataloader = experiment_train_loader,
                      test_dataloader = experiment_val_loader,
                      loss_function = criterion,
                      optimizer = optimizer,
                      metric = "accuracy",
                      epochs = epoch,
                      device = device,
                      writer = utils.create_writer(experiment_name = f"{epoch}_epochs",
                                                   model_name = extractor.__class__.__name__),
                      verbose = 2)

    elif extractor == DENSENET:

      model, transformer = extractors.create_densenet121(output_shape = len(class_names))
      train_data.transform = transformer
      test_data.transform = transformer
      experiment_train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                                            batch_size = 32,
                                                            shuffle = True,
                                                            num_workers = os.cpu_count(),
                                                            pin_memory = True)
      experiment_val_loader = torch.utils.data.DataLoader(dataset = test_data,
                                                          batch_size = 32,
                                                          shuffle = False,
                                                          num_workers = os.cpu_count(),
                                                          pin_memory = True)

      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001)

      model_ops.train(model = model,
                      train_dataloader = experiment_train_loader,
                      test_dataloader = experiment_val_loader,
                      loss_function = criterion,
                      optimizer = optimizer,
                      metric = "accuracy",
                      epochs = epoch,
                      device = device,
                      writer = utils.create_writer(experiment_name = f"{epoch}_epochs",
                                                   model_name = extractor.__class__.__name__),
                      verbose = 2)

    else:

      model, transformer = extractors.create_resnet50(output_shape = len(class_names))
      train_data.transform = transformer
      test_data.transform = transformer
      experiment_train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                                            batch_size = 32,
                                                            shuffle = True,
                                                            num_workers = os.cpu_count(),
                                                            pin_memory = True)
      experiment_val_loader = torch.utils.data.DataLoader(dataset = test_data,
                                                          batch_size = 32,
                                                          shuffle = False,
                                                          num_workers = os.cpu_count(),
                                                          pin_memory = True)

      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001)

      model_ops.train(model = model,
                      train_dataloader = experiment_train_loader,
                      test_dataloader = experiment_val_loader,
                      loss_function = criterion,
                      optimizer = optimizer,
                      metric = "accuracy",
                      epochs = epoch,
                      device = device,
                      writer = utils.create_writer(experiment_name = f"{epoch}_epochs",
                                                   model_name = extractor.__class__.__name__),
                      verbose = 2)

Experiment n°1, model: AlexNet, 5 epochs
[INFO] Created SummaryWriter, saving to: runs/2023-11-26/5_epochs/AlexNet...


Training:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 01: Train Loss: 2.282 | Train Accuracy: 43.88% | Test Loss: 1.806 | Test Accuracy: 53.08%
Epoch 02: Train Loss: 1.252 | Train Accuracy: 65.32% | Test Loss: 1.833 | Test Accuracy: 53.75%
Epoch 03: Train Loss: 0.820 | Train Accuracy: 77.33% | Test Loss: 1.938 | Test Accuracy: 53.40%
Epoch 04: Train Loss: 0.556 | Train Accuracy: 85.31% | Test Loss: 2.070 | Test Accuracy: 53.13%
Epoch 05: Train Loss: 0.395 | Train Accuracy: 90.06% | Test Loss: 2.243 | Test Accuracy: 52.45%
Experiment n°2, model: VGG, 5 epochs
[INFO] Created SummaryWriter, saving to: runs/2023-11-26/5_epochs/VGG...


Training:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 01: Train Loss: 2.047 | Train Accuracy: 46.48% | Test Loss: 1.555 | Test Accuracy: 56.81%
Epoch 02: Train Loss: 0.789 | Train Accuracy: 78.01% | Test Loss: 1.575 | Test Accuracy: 56.75%
Epoch 03: Train Loss: 0.387 | Train Accuracy: 91.28% | Test Loss: 1.625 | Test Accuracy: 57.20%
Epoch 04: Train Loss: 0.192 | Train Accuracy: 97.29% | Test Loss: 1.688 | Test Accuracy: 57.70%
Epoch 05: Train Loss: 0.102 | Train Accuracy: 99.30% | Test Loss: 1.744 | Test Accuracy: 58.22%
Experiment n°3, model: DenseNet, 5 epochs
[INFO] Created SummaryWriter, saving to: runs/2023-11-26/5_epochs/DenseNet...


...

Epoch 01: Train Loss: 2.309 | Train Accuracy: 42.89% | Test Loss: 1.684 | Test Accuracy: 54.64%
Epoch 02: Train Loss: 1.602 | Train Accuracy: 56.64% | Test Loss: 1.574 | Test Accuracy: 57.07%
Epoch 03: Train Loss: 1.449 | Train Accuracy: 60.16% | Test Loss: 1.552 | Test Accuracy: 57.38%
Epoch 04: Train Loss: 1.360 | Train Accuracy: 62.31% | Test Loss: 1.555 | Test Accuracy: 57.50%
Epoch 05: Train Loss: 1.299 | Train Accuracy: 63.88% | Test Loss: 1.566 | Test Accuracy: 57.43%
Experiment n°4, model: ResNet, 5 epochs
[INFO] Created SummaryWriter, saving to: runs/2023-11-26/5_epochs/ResNet...


Training:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 01: Train Loss: 2.367 | Train Accuracy: 42.66% | Test Loss: 1.735 | Test Accuracy: 55.00%
Epoch 02: Train Loss: 1.520 | Train Accuracy: 59.69% | Test Loss: 1.573 | Test Accuracy: 57.41%
Epoch 03: Train Loss: 1.252 | Train Accuracy: 66.67% | Test Loss: 1.526 | Test Accuracy: 58.29%
Epoch 04: Train Loss: 1.074 | Train Accuracy: 71.66% | Test Loss: 1.515 | Test Accuracy: 58.62%
Epoch 05: Train Loss: 0.939 | Train Accuracy: 75.58% | Test Loss: 1.521 | Test Accuracy: 58.75%
Experiment n°5, model: AlexNet, 10 epochs
[INFO] Created SummaryWriter, saving to: runs/2023-11-26/10_epochs/AlexNet...


Training:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 01: Train Loss: 2.282 | Train Accuracy: 43.88% | Test Loss: 1.806 | Test Accuracy: 53.08%
Epoch 02: Train Loss: 1.252 | Train Accuracy: 65.32% | Test Loss: 1.833 | Test Accuracy: 53.75%
Epoch 03: Train Loss: 0.820 | Train Accuracy: 77.33% | Test Loss: 1.938 | Test Accuracy: 53.40%
Epoch 04: Train Loss: 0.556 | Train Accuracy: 85.31% | Test Loss: 2.070 | Test Accuracy: 53.13%
Epoch 05: Train Loss: 0.395 | Train Accuracy: 90.06% | Test Loss: 2.243 | Test Accuracy: 52.45%
Epoch 06: Train Loss: 0.296 | Train Accuracy: 92.75% | Test Loss: 2.336 | Test Accuracy: 53.30%
Epoch 07: Train Loss: 0.239 | Train Accuracy: 93.81% | Test Loss: 2.472 | Test Accuracy: 53.31%
Epoch 08: Train Loss: 0.196 | Train Accuracy: 94.74% | Test Loss: 2.580 | Test Accuracy: 53.72%


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs